# HW5 Codes

In [2]:
%pip install --quiet pandas numpy altair vega_datasets

import pandas as pd
import numpy as np
import altair as alt

Note: you may need to restart the kernel to use updated packages.


### Load the data

In [3]:
illinois_url = 'https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv'

In [4]:
illinois = pd.read_csv(illinois_url)

In [8]:
illinois.columns

Index(['_id', 'License Type', 'Description', 'License Number',
       'License Status', 'Business', 'Title', 'First Name', 'Middle',
       'Last Name', 'Prefix', 'Suffix', 'Business Name', 'BusinessDBA',
       'Original Issue Date', 'Effective Date', 'Expiration Date', 'City',
       'State', 'Zip', 'County', 'Specialty/Qualifier',
       'Controlled Substance Schedule',
       'Delegated Controlled Substance Schedule', 'Ever Disciplined',
       'LastModifiedDate', 'Case Number', 'Action', 'Discipline Start Date',
       'Discipline End Date', 'Discipline Reason'],
      dtype='object')

In [7]:
sorted(illinois["License Status"].dropna().unique())

['ACTIVE',
 'CANCELLED',
 'CHANGE OF OWNERSHIP',
 'CLOSED',
 'DECEASED',
 'EXPIRED',
 'INACTIVE',
 'INOPERATIVE',
 'NOT RENEWED',
 'Non Sufficient Fund Check Terminated',
 'TERMINATED CARD RETURNED',
 'TERMINATED VALID REASON',
 'TERMINATED WITHOUT CARD']

In [18]:
illinois.isna().mean().round(3) * 100

_id                                          0.0
License Type                                 0.0
Description                                  0.0
License Number                               0.6
License Status                               0.0
Business                                     0.0
Title                                       98.9
First Name                                   4.0
Middle                                      63.8
Last Name                                    4.0
Prefix                                     100.0
Suffix                                      95.9
Business Name                                0.0
BusinessDBA                                 98.8
Original Issue Date                          0.0
Effective Date                               7.9
Expiration Date                              5.0
City                                         0.1
State                                        0.0
Zip                                          0.7
County              

In [ ]:
Licence_counts = (
    illinois.groupby("License Type")
    .size()
    .reset_index(name="Count")
    .sort_values("Count", ascending=False)
)


chart1 = alt.Chart(Licence_counts).mark_bar().encode(
    x=alt.X("Count:Q", title="Number of Licenses"),
    y=alt.Y("License Type:N", sort='-x', title="License Type"),
    tooltip=["License Type", "Count"]
    ).properties(
        width=600,
        height=400,
        title="License Type Distribution"
    )

chart1

alt.Chart(...)

In [ ]:
# Summary table
disc = (illinois.groupby(["License Type", "Ever Disciplined"]).size().reset_index(name="Count"))

# Top 20 License Types
top_types = disc.groupby("License Type")["Count"].sum().nlargest(20).index
disc = disc[disc["License Type"].isin(top_types)]

In [ ]:
# Rate of discipline 
disc['Disc_Rate'] = (
    disc['Count'] / disc.groupby('License Type')['Count'].transform('sum')
)

# Re-order License Types by discipline rate  (I got help from ChatGPT here with prompt like 'how to get rates per groups and order it in python')
sort_order = (
    disc.loc[disc['Ever Disciplined'] == 'Y']
        .groupby('License Type')['Disc_Rate']
        .sum()
        .sort_values(ascending=False)
        .index.tolist()
)

In [80]:
chart2 = alt.Chart(disc).mark_bar().encode(
    y = alt.Y("License Type:N", sort = sort_order, title = "License Type"),
    x = alt.X("Disc_Rate:Q", title = "Rate of Ever Disciplined"),  
    color = alt.Color("Ever Disciplined:N", title = "Disciplined?"),
    tooltip = ["License Type", "Ever Disciplined", "Count",
             alt.Tooltip("Disc_Rate:Q")]
).properties(
    width=600,
    height=300,
    title="License Type vs. Ever Disciplined (Proportion, Sorted by Discipline Rate)"
)

chart2

alt.Chart(...)

In [81]:
myJekyllDir = '/Users/Haki/Documents/GitHub/hopeintheair.github.io/assets/json/'
chart1.save(myJekyllDir + 'license.json')
chart2.save(myJekyllDir + 'Disciplined.json')

In [ ]:
#myCsvDir = '/Users/Haki/Documents/GitHub/hopeintheair.github.io/assets/csv/'
#chartURL.save(myJekyllDir + 'altair_mobilityURL.json')

#illinois.to_csv("/Users/Haki/Documents/GitHub/hopeintheair.github.io/assets/csv/licenses_fall2022.csv", index=True)